### NEURAL NETWORKS

Before diving into the following content, make sure you are familiar with convolutional neural network. A good tutorial can be found here: https://cs231n.github.io/convolutional-networks/

In [ ]:
"""neural network
-神经网络可以通过torch.nn来搭建
-一个nn.Module包含了layers，以及forward(input)方法，该方法返回output
-一个典型的神经网络的搭建及训练过程如下：
 * Define the neural network that has some learnable parameters (or weights)
 * Iterate over a dataset of inputs
 * Process input through the network
 * Compute the loss (how far is the output from being correct)
 * Propagate gradients back into the network’s parameters
 * Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

"""

In [ ]:
# define the network
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 1 input image, 6 output channels, 3*3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)  # nn.Conv2d will take in a 4D Tensor of nSamples*nChannels*Height*Width
        self.conv2 = nn.Conv2d(6, 16, 5)  # input.unsqueeeze(0) will add a fake batch dimension to a single sample
        # an affine operation: y = Wx+b
        self.fc1 = nn.Linear(16*5*5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # torch.nn only support mini-batches, not a single sample
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)      

In [ ]:
params = list(net.parameters())  
print(len(params))  # 打印参数数目
print(params[0].size())

In [ ]:
image = torch.randn(1,1,32,32)
result = net(image)
print(result)

In [ ]:
net.zero_grad()
result.backward(torch.randn(1,10))

In [ ]:
"""Loss Function
-nn包中有很多损失函数，最简单的是nn.MSELoss
"""

In [ ]:
target = torch.randn(1,10)
criterion = nn.MSELoss()

loss = criterion(result, target)
print(loss)

In [ ]:
"""Backprop
- 在做backward之前，要清空之前的梯度，否则梯度就会累加
"""

In [ ]:
net.zero_grad()
print(net.conv1.bias.grad)

loss.backward()

print(net.conv1.bias.grad)

In [ ]:
"""Update the weight
-weight = weight - learning_rate * gradient
-torch.optim中实现了多种梯度加速算法
"""

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()
output = net(image)
loss = criterion(output, target)
loss.backward()
optimizer.step()